In [5]:
from plotly import figure_factory as ff
from icalendar import Calendar
import datetime
import plotly.io as pio
pio.renderers.default = "notebook"
import requests
import os
import pandas as pd

In [6]:
def baixar_calendario_OTA(url, filename):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check if the request was successful
        
        # Create 'calendars' directory if it doesn't exist
        if not os.path.exists('calendars'):
            os.makedirs('calendars')
            
        # Save the calendar file
        filepath = os.path.join('calendars', filename)
        with open(filepath, 'wb') as f:
            f.write(response.content)
        #print(f"Successfully saved {filename}")
    except Exception as e:
        print(f"Error downloading {filename}: {str(e)}")


In [7]:
def atualizar_summaries_ical(ical_file_path):
    """
    Abre um arquivo iCal, atualiza os campos 'SUMMARY' de acordo com
    regras predefinidas e salva o arquivo (sobrescrevendo o original).

    Args:
        ical_file_path (str): O caminho para o arquivo .ics a ser modificado.
    """
    
    print(f"Processando arquivo: {ical_file_path}")

    # --- 1. Definir as regras de substituição ---
    regras_summary = {
        'CLOSED - Not available': 'Booking',
        'Airbnb (Not available)': 'Direto',
        'Reserved': 'Airbnb'
    }

    # --- 2. Ler o arquivo iCal ---
    try:
        with open(ical_file_path, 'rb') as f:
            cal = Calendar.from_ical(f.read())
    except FileNotFoundError:
        print(f"  ERRO: Arquivo '{ical_file_path}' não encontrado.")
        return
    except Exception as e:
        print(f"  ERRO: Falha ao ler o arquivo iCal: {e}")
        return

    mudancas_feitas = 0
    total_eventos = 0

    # --- 3. Iterar e aplicar as regras ---
    for component in cal.walk('VEVENT'):
        total_eventos += 1
        
        # Usar str() para garantir que é uma string
        summary_original = str(component.get('summary')) 

        # Verifica se o summary atual está na nossa lista de regras
        if summary_original in regras_summary:
            novo_summary = regras_summary[summary_original]
            
            # Atualiza o summary no componente
            component['summary'] = novo_summary
            
            #print(f"  -> Atualizado: '{summary_original}' -> '{novo_summary}'")
            mudancas_feitas += 1

    # --- 4. Salvar o arquivo (apenas se houver mudanças) ---
    if mudancas_feitas > 0:
        print(f"Total de {mudancas_feitas} de {total_eventos} eventos atualizados.")
        try:
            with open(ical_file_path, 'wb') as f:
                # .to_ical() retorna bytes, por isso 'wb' (write bytes)
                f.write(cal.to_ical())
            print(f"Arquivo '{ical_file_path}' salvo com sucesso.\n")
        except Exception as e:
            print(f"  ERRO: Falha ao salvar o arquivo iCal: {e}")
    else:
        print(f"Nenhuma atualização necessária para '{ical_file_path}'. ({total_eventos} eventos verificados)")

In [8]:
#Ap101 Airbnb
ap101_airbnb = "https://www.airbnb.com.br/calendar/ical/2710524.ics?s=bc7ceeb248c20334bd52ccf30acabc07"

#Ap201 Airbnb
ap201_airbnb = "https://www.airbnb.com.br/calendar/ical/17246144.ics?s=466804a1ee4a8d8dbf117f6fc6bd1420"

#C108 Airbnb
c108_airbnb = "https://www.airbnb.com.br/calendar/ical/1428938746995365808.ics?s=a66ee2191a8f4d5486cb078f9de34f88"

#C108 Booking
c108_booking = "https://ical.booking.com/v1/export?t=359f0248-5265-4cde-82a8-c353156ad9be"

#CBL004 Airbnb
cbl004_airbnb = "https://www.airbnb.com.br/calendar/ical/35679659.ics?s=bc17a6ccf8f6e21d4da62b155265b96c"

#CBL004 Booking
cbl004_booking = "https://admin.booking.com/hotel/hoteladmin/ical.html?t=2e033af1-32e7-4f9c-aabe-afbfd5ece986"

#D014 Airbnb
d014_airbnb = "https://www.airbnb.com.br/calendar/ical/1447417239837262921.ics?s=d025b75182c4a9391da767c47ee15692"

#D014 Booking
# anuncio velho d014_booking = "https://ical.booking.com/v1/export?t=5880d0d9-ed83-4ca6-a781-62738ace861e"
#  novo         
d014_booking =  "https://ical.booking.com/v1/export?t=d980d943-1019-4765-b8b9-670a2439cf46"

#F216 Airbnb
f216_airbnb = "https://www.airbnb.com/calendar/ical/1470964694636236725.ics?s=95cf86fef315ea900c0ed146f223abf7"

#F216 Booking
f216_booking = ""

#Feriados no Brasil
feriados_brasil = "https://www.calendario.com.br/feriados/brasil.ics" #https://calendar.google.com/calendar/ical/pt.brazilian%23holiday%40group.v.calendar.google.com/public/basic.ics


# Dictionary of calendars to download
calendars = {
    'ap101_airbnb.ics': ap101_airbnb,
    'ap201_airbnb.ics': ap201_airbnb,
    'c108_airbnb.ics': c108_airbnb,
    'c108_booking.ics': c108_booking,
    'cbl004_airbnb.ics': cbl004_airbnb,
    'cbl004_booking.ics': cbl004_booking,
    'd014_airbnb.ics': d014_airbnb,
    'd014_booking.ics': d014_booking,
    'f216_airbnb.ics': f216_airbnb,
    'f216_booking.ics': f216_booking,
    #'feriados_brasil.ics': feriados_brasil
}

# Download all calendars
for filename, url in calendars.items():
    baixar_calendario_OTA(url, filename)
    atualizar_summaries_ical(os.path.join('calendars', filename))

Processando arquivo: calendars\ap101_airbnb.ics
Total de 20 de 20 eventos atualizados.
Arquivo 'calendars\ap101_airbnb.ics' salvo com sucesso.

Processando arquivo: calendars\ap201_airbnb.ics
Total de 26 de 26 eventos atualizados.
Arquivo 'calendars\ap201_airbnb.ics' salvo com sucesso.

Processando arquivo: calendars\c108_airbnb.ics
Total de 7 de 7 eventos atualizados.
Arquivo 'calendars\c108_airbnb.ics' salvo com sucesso.

Processando arquivo: calendars\c108_booking.ics
Total de 3 de 3 eventos atualizados.
Arquivo 'calendars\c108_booking.ics' salvo com sucesso.

Processando arquivo: calendars\cbl004_airbnb.ics
Total de 14 de 14 eventos atualizados.
Arquivo 'calendars\cbl004_airbnb.ics' salvo com sucesso.

Processando arquivo: calendars\cbl004_booking.ics
Total de 1 de 1 eventos atualizados.
Arquivo 'calendars\cbl004_booking.ics' salvo com sucesso.

Processando arquivo: calendars\d014_airbnb.ics
Total de 5 de 5 eventos atualizados.
Arquivo 'calendars\d014_airbnb.ics' salvo com sucesso.